In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time

# load the dataset
dataset = pd.read_csv('./data/new_train.csv')
#dataset = dataset.drop(columns=['Page content', 'article', 'title', 'article_clean', 'title_clean', 'Unnamed: 0', 'dummy', 'tfidf'])
dataset.head()

FileNotFoundError: [Errno 2] File b'./data/new_train.csv' does not exist: b'./data/new_train.csv'

In [84]:
dataset2 = pd.read_csv('./data/cat_weight.csv')
dataset2 = dataset2.drop(columns=['Page content', 'Unnamed: 0'])
dataset2.head()

,Id,Popularity,cat_weight
0,0,-1,-0.707606
1,1,1,0.461590
2,2,1,0.248302
3,3,-1,-0.106637
4,4,-1,-0.141015


In [85]:
dataset3 = pd.read_csv('./data/train_article_weight.csv')
dataset3 = dataset3.drop(columns=['Page content', 'Unnamed: 0'])
dataset3.head()

,Id,Popularity,article_weight
0,0,-1,-25.934944
1,1,1,23.763097
2,2,1,-21.804157
3,3,-1,-4.107853
4,4,-1,3.021477


In [86]:
dataset['cat_weight'] = dataset2['cat_weight']
dataset['article_weight'] = dataset3['article_weight']

In [87]:
dataset = dataset.loc[20000:, :]
dataset.head()

,Popularity,is_weekend,is_mon,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total,cat_weight,article_weight
20000,-1,0,0,2629,455,48,0.620536,0.513761,0.054945,14,12,26,-0.838750,-9.913167
20001,-1,0,1,775,140,16,0.828358,0.558333,0.064286,1,11,12,-1.090259,-1.711027
20002,1,0,0,802,143,14,0.720588,0.525097,0.202797,1,8,9,0.188553,3.010395
20003,-1,1,0,792,123,11,0.786325,0.559809,0.097561,2,22,24,0.011343,5.608189
20004,-1,0,1,3428,573,67,0.596120,0.538462,0.022688,13,42,55,0.230868,-7.406163


In [88]:
X = dataset.drop(columns=['Popularity'])
y = dataset['Popularity']
# for i in range(dataset.shape[0]):
#     tmp = []
#     tmp.append(dataset['title_freq'][i])
#     tmp.append(dataset['weekday'][i])
#     tmp.append(dataset['unique'][i])
#     tmp.append(dataset['word_count'][i])
#     #tmp.extend([float(x) for x in dataset['keywords'][0][1:-1].split(',')])
#     #tmp.extend([int(x) for x in dataset['category'][0][1:-1].split(' ')])
#     X.append(tmp)
#     y.append(dataset['Popularity'][i])

,is_weekend,is_mon,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total,cat_weight,article_weight
20000,0,0,2629,455,48,0.620536,0.513761,0.054945,14,12,26,-0.838750,-9.913167
20001,0,1,775,140,16,0.828358,0.558333,0.064286,1,11,12,-1.090259,-1.711027
20002,0,0,802,143,14,0.720588,0.525097,0.202797,1,8,9,0.188553,3.010395
20003,1,0,792,123,11,0.786325,0.559809,0.097561,2,22,24,0.011343,5.608189
20004,0,1,3428,573,67,0.596120,0.538462,0.022688,13,42,55,0.230868,-7.406163
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27638,0,0,1037,170,15,0.731707,0.532468,0.076471,2,12,14,-0.835429,6.522667
27639,0,0,1186,193,12,0.702703,0.560606,0.077720,3,23,26,-0.734746,3.222749
27640,0,0,680,110,10,0.676190,0.317221,0.090909,15,23,38,-0.840086,-16.933464
27641,0,0,1605,254,23,0.704453,0.566514,0.066929,3,17,20,-0.415931,-8.885657


In [90]:
from sklearn.model_selection import train_test_split
# X = np.array(X)
# y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [120]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import StandardScaler

# param_C = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]

# #clf = LogisticRegression(random_state=0)
# pipe3 = Pipeline([['sc', StandardScaler(), 
#                   ('clf', LogisticRegression())]])

# # set the param_grid parameter of GridSearchCV to a list of dictionaries
# param_grid = [{'clf__C': param_C,
#                'clf__solver': ['liblinear']}]
# gs = GridSearchCV(estimator=pipe3, 
#                   param_grid=param_grid, 
#                   scoring='accuracy')

# gs = gs.fit(X_train, y_train)
# print(gs.best_score_)
# print(gs.best_params_)

best_k, best_score = -1, -1
clfs = {}

for k in [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]: # experiment different hyperparameter
    pipe = Pipeline([['sc', StandardScaler()], ['clf', LogisticRegression(C=k)]])
    pipe.fit(X_train, y_train)
    # K-Fold CV
    scores = cross_val_score(pipe, X_train, y_train, cv=5)
    print('[%f-C]\nValidation accuracy: %.3f %s' % (k, scores.mean(), scores))
    if scores.mean() > best_score:
        best_k, best_score = k, scores.mean()
    clfs[k] = pipe

C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

[0.010000-C]
Validation accuracy: 0.543 [0.51921504 0.53638594 0.5478332  0.56173344 0.54991817]
[0.100000-C]
Validation accuracy: 0.544 [0.52003271 0.5372036  0.55028618 0.56336877 0.55155483]
[1.000000-C]
Validation accuracy: 0.542 [0.52330335 0.53638594 0.54865086 0.56173344 0.5400982 ]


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

[10.000000-C]
Validation accuracy: 0.543 [0.52575634 0.54374489 0.54701554 0.56173344 0.53518822]
[100.000000-C]
Validation accuracy: 0.542 [0.52493868 0.54292723 0.54701554 0.56091578 0.53518822]


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

[1000.000000-C]
Validation accuracy: 0.542 [0.52493868 0.54210957 0.54701554 0.56091578 0.53518822]
[10000.000000-C]
Validation accuracy: 0.542 [0.52493868 0.54210957 0.54701554 0.56091578 0.53518822]


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [116]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
best_k, best_score = -1, -1
clfs = {}

for k in [1, 15, 50, 100, 1000, 1500]: # experiment different hyperparameter
    pipe = Pipeline([['sc', StandardScaler()], ['clf', KNeighborsClassifier(n_neighbors=k)]])
    pipe.fit(X_train, y_train)
    # K-Fold CV
    scores = cross_val_score(pipe, X_train, y_train, cv=5)
    print('[%d-NN]\nValidation accuracy: %.3f %s' % (k, scores.mean(), scores))
    if scores.mean() > best_score:
        best_k, best_score = k, scores.mean()
    clfs[k] = pipe

[1-NN]
Validation accuracy: 0.511 [0.5053148  0.51349141 0.50940311 0.50858545 0.51963993]
[15-NN]
Validation accuracy: 0.517 [0.51839738 0.51267375 0.51757972 0.53475061 0.50409165]
[50-NN]
Validation accuracy: 0.527 [0.52739166 0.52330335 0.54374489 0.526574   0.51391162]
[100-NN]
Validation accuracy: 0.529 [0.52412101 0.53147997 0.53229763 0.53556827 0.51963993]
[1000-NN]
Validation accuracy: 0.540 [0.54456255 0.52984464 0.54456255 0.54210957 0.53846154]
[1500-NN]
Validation accuracy: 0.540 [0.55028618 0.52820932 0.54619787 0.53475061 0.5400982 ]


In [123]:
from sklearn.tree import DecisionTreeClassifier
best_k, best_score = -1, -1
clfs = {}

for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 50, 100, 1000, 1500]: # experiment different hyperparameter
    pipe = Pipeline([['sc', StandardScaler()], ['clf', DecisionTreeClassifier(max_depth = k, random_state = 0)]])
    pipe.fit(X_train, y_train)
    # K-Fold CV
    scores = cross_val_score(pipe, X_train, y_train, cv=5)
    print('[%d-depth]\nValidation accuracy: %.3f %s' % (k, scores.mean(), scores))
    if scores.mean() > best_score:
        best_k, best_score = k, scores.mean()
    clfs[k] = pipe


[1-depth]
Validation accuracy: 0.538 [0.5478332  0.52330335 0.54701554 0.53883892 0.53355155]
[2-depth]
Validation accuracy: 0.536 [0.53965658 0.52330335 0.54456255 0.53883892 0.53355155]
[3-depth]
Validation accuracy: 0.537 [0.53802126 0.52739166 0.54701554 0.5478332  0.52291326]
[4-depth]
Validation accuracy: 0.535 [0.53556827 0.53556827 0.53802126 0.54292723 0.52291326]
[5-depth]
Validation accuracy: 0.531 [0.52085037 0.52493868 0.54374489 0.53883892 0.52864157]
[6-depth]
Validation accuracy: 0.521 [0.52330335 0.52984464 0.52166803 0.51921504 0.50981997]
[7-depth]
Validation accuracy: 0.520 [0.49141455 0.52902698 0.53066231 0.53965658 0.51145663]
[8-depth]
Validation accuracy: 0.527 [0.53393295 0.53229763 0.5372036  0.52493868 0.50490998]
[9-depth]
Validation accuracy: 0.525 [0.52330335 0.52575634 0.52902698 0.54047424 0.50572831]
[10-depth]
Validation accuracy: 0.519 [0.51839738 0.51757972 0.526574   0.54047424 0.49099836]
[15-depth]
Validation accuracy: 0.498 [0.473426   0.526574 

In [183]:
#from sklearn.tree import DecisionTreeClassifier
best_k, best_score = -1, -1
clfs = {}

for k in [1, 100, 500, 1000, 1500]: # experiment different hyperparameter
    pipe = Pipeline([['sc', StandardScaler()], ['clf', RandomForestClassifier(n_estimators=k, random_state=1, n_jobs=4)]])
    pipe.fit(X_train, y_train)
    # K-Fold CV
    scores = cross_val_score(pipe, X_train, y_train, cv=5)
    print('[%d-depth]\nValidation accuracy: %.3f %s' % (k, scores.mean(), scores))
    if scores.mean() > best_score:
        best_k, best_score = k, scores.mean()
    clfs[k] = pipe

[1-depth]
Validation accuracy: 0.508 [0.49877351 0.49304988 0.51349141 0.52248569 0.50981997]
[100-depth]
Validation accuracy: 0.521 [0.51267375 0.53229763 0.51185609 0.5372036  0.50900164]
[500-depth]
Validation accuracy: 0.531 [0.53147997 0.52739166 0.52575634 0.53638594 0.53191489]
[1000-depth]
Validation accuracy: 0.532 [0.53066231 0.53638594 0.51921504 0.53638594 0.53927987]
[1500-depth]
Validation accuracy: 0.531 [0.53066231 0.53066231 0.52739166 0.53147997 0.53436989]


In [121]:
from sklearn.svm import SVC

param_C = [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
param_gamma = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]

svm = SVC(random_state=0)

# set the param_grid parameter of GridSearchCV to a list of dictionaries
param_grid = [{'C': param_C, 
               'gamma': param_gamma, 
               'kernel': ['rbf']}]
gs = GridSearchCV(estimator=svm, 
                  param_grid=param_grid, 
                  scoring='accuracy')

gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.5188092901537456
{'C': 10000.0, 'gamma': 1e-05, 'kernel': 'rbf'}


In [159]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=1)
# single decision tree
tree = tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

tree_train = accuracy_score(y_train, y_train_pred)
tree_test = accuracy_score(y_test, y_test_pred)
print('[DecisionTree] accuracy-train = %.3f, accuracy-test = %.3f' % 
      (tree_train, tree_test))

# adaboost
ada = AdaBoostClassifier(base_estimator=tree, n_estimators=100)
ada = ada.fit(X_train, y_train)
y_train_pred = ada.predict(X_train)
y_test_pred = ada.predict(X_test)

ada_train = accuracy_score(y_train, y_train_pred)
ada_test = accuracy_score(y_test, y_test_pred)
print('[AdaBoost] accuracy-train = %.3f, accuracy-test = %.3f' % 
      (ada_train, ada_test))

[DecisionTree] accuracy-train = 0.538, accuracy-test = 0.528
[AdaBoost] accuracy-train = 0.596, accuracy-test = 0.530


In [199]:
pipe_lr =best_vt
bag = BaggingClassifier(base_estimator=best_vt, n_estimators=100, 
                        max_samples=0.7, bootstrap=True,
                        max_features=1, bootstrap_features=False, 
                        n_jobs=4, random_state=1)

In [200]:
# single DecisionTree
tree = pipe_lr.fit(X_train, y_train)
y_train_pred = pipe_lr.predict(X_train)
y_test_pred = pipe_lr.predict(X_test)

tree_train = accuracy_score(y_train, y_train_pred)
tree_test = accuracy_score(y_test, y_test_pred)
print('[DecisionTree] accuracy-train = %.3f, accuracy-test = %.3f' % (tree_train, tree_test))
# Bagging
bag = bag.fit(X_train, y_train)
y_train_pred = bag.predict(X_train)
y_test_pred = bag.predict(X_test)

bag_train = accuracy_score(y_train, y_train_pred) 
bag_test = accuracy_score(y_test, y_test_pred) 
print('[Bagging] auc-train = %.3f, auc-test = %.3f' % (bag_train, bag_test))

C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[DecisionTree] accuracy-train = 0.933, accuracy-test = 0.526


ValueError: Pipeline.fit does not accept the sample_weight parameter. You can pass parameters to specific steps of your pipeline using the stepname__parameter format, e.g. `Pipeline.fit(X, y, logisticregression__sample_weight=sample_weight)`.

In [182]:
#rnd_clf = RandomForestClassifier()
pipe_rc = Pipeline([['sc', StandardScaler()], ['clf', RandomForestClassifier()]])
pipe_rc.fit(X_train, y_train)

y_pred = pipe_rc.predict(X_test)
print('Accuracy (forest): %.3f' % accuracy_score(y_test, y_pred))

Accuracy (forest): 0.509


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# GOGOGOGOGO

In [185]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier

names = ['DecisionTreeClassifier', 
         'KNeighborsClassifier',
         'LogisticRegression']

pipe1 = Pipeline([['sc', StandardScaler()], ['clf', DecisionTreeClassifier(max_depth = 3, random_state = 0)]])
pipe2 = Pipeline([['sc', StandardScaler()], ['clf', KNeighborsClassifier(n_neighbors=1000)]])
pipe3 = Pipeline([['sc', StandardScaler()], ['clf', LogisticRegression(C=0.1)]])
# pipe3 = Pipeline([['sc', StandardScaler()], ['clf', BaggingClassifier(base_estimator=pipe_lr, n_estimators=500, 
#                         max_samples=0.7, bootstrap=True,
#                         max_features=1, bootstrap_features=True, 
#                         n_jobs=4, random_state=1)]])
pipe4 = Pipeline([['sc', StandardScaler()], ['clf', RandomForestClassifier(n_estimators=1000, random_state=1, n_jobs=4)]])


print('[Voting]')
best_vt, best_w, best_score = None, (), -1
for a, b, c, d in list(itertools.permutations(range(0,4))): # try some weight combination
    clf = VotingClassifier(estimators=[('DecisionTreeClassifier', pipe1), ('KNeighborsClassifier', pipe2), 
                                       ('LogisticRegression', pipe3), ('RandomForestClassifier', pipe4)], 
                           voting='soft', weights=[a,b,c,d])
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=10, scoring='roc_auc')
    print('%s: %.3f (+/- %.3f)' % ((a,b,c,d), scores.mean(), scores.std()))
    if best_score < scores.mean():
        best_vt, best_w, best_score = clf, (a, b, c, d), scores.mean()

print('\nBest %s: %.3f' % (best_w, best_score))

[Voting]


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

(0, 1, 2, 3): 0.554 (+/- 0.019)


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

(0, 1, 3, 2): 0.559 (+/- 0.018)


...

(3, 1, 2, 0): 0.558 (+/- 0.016)


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

(3, 2, 0, 1): 0.550 (+/- 0.021)


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

(3, 2, 1, 0): 0.553 (+/- 0.017)

Best (0, 2, 3, 1): 0.562


In [187]:
best_vt.fit(X_train, y_train)

y_pred = best_vt.predict(X_train)
print('Accuracy (forest): %.2f' % accuracy_score(y_train, y_pred))

y_pred = best_vt.predict(X_test)
print('Accuracy (forest): %.3f' % accuracy_score(y_test, y_pred))

C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy (forest): 0.93
Accuracy (forest): 0.526


In [149]:
pipe_lr = Pipeline([['sc', StandardScaler()], ['clf', LogisticRegression(C=0.1)]])
pipe_lr.fit(X_train, y_train)

y_pred = pipe_lr.predict(X_test)
print('Accuracy (forest): %.3f' % accuracy_score(y_test, y_pred))

Accuracy (forest): 0.532


C:\Users\yuwei\.conda\envs\tf2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# TEST DATA

In [188]:
dataset = pd.read_csv('./data/test_3.csv')
dataset = dataset.drop(columns=['Page content', 'article', 'title', 'article_clean', 'title_clean', 'Unnamed: 0', 'dummy', 'tfidf'])
dataset.head()

,is_weekend,is_mon,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total
0,0,1,2294,382,37,0.704301,0.590476,0.075916,1,30,31
1,0,0,489,84,6,0.696203,0.530201,0.178571,3,13,16
2,0,0,620,105,13,0.808081,0.589286,0.142857,2,13,15
3,0,0,670,113,7,0.787037,0.666667,0.097345,1,15,16
4,0,0,835,146,10,0.839416,0.608889,0.102740,1,10,11


In [189]:
dataset2 = pd.read_csv('./data/test_cat_weight.csv')
dataset2 = dataset2.drop(columns=['Page content', 'Unnamed: 0'])
dataset2.head()

,Id,cat_weight
0,27643,-0.514820
1,27644,-0.791362
2,27645,0.129010
3,27646,-1.407612
4,27647,-0.690529


In [190]:
dataset3 = pd.read_csv('./data/train_article_weight.csv')
dataset3 = dataset3.drop(columns=['Page content', 'Unnamed: 0'])
dataset3.head()

,Id,Popularity,article_weight
0,0,-1,-25.934944
1,1,1,23.763097
2,2,1,-21.804157
3,3,-1,-4.107853
4,4,-1,3.021477


In [191]:
dataset['cat_weight'] = dataset2['cat_weight']
dataset['article_weight'] = dataset3['article_weight']

In [192]:
dataset.head()

,is_weekend,is_mon,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of total,cat_weight,article_weight
0,0,1,2294,382,37,0.704301,0.590476,0.075916,1,30,31,-0.514820,-25.934944
1,0,0,489,84,6,0.696203,0.530201,0.178571,3,13,16,-0.791362,23.763097
2,0,0,620,105,13,0.808081,0.589286,0.142857,2,13,15,0.129010,-21.804157
3,0,0,670,113,7,0.787037,0.666667,0.097345,1,15,16,-1.407612,-4.107853
4,0,0,835,146,10,0.839416,0.608889,0.102740,1,10,11,-0.690529,3.021477


In [193]:
y_pred = best_vt.predict(np.array(dataset))

In [194]:
res = pd.DataFrame(y_pred)
#res.columns(['Popularity'])
ID = [i for i in range(27643, 27643 + res.shape[0])]
res['Id'] = ID
res.columns = ['Popularity', 'Id']
res = res.set_index('Id')
#res.columns(['Id', 'Popularity'])

In [195]:
res.head(20)

,Popularity
Id,
27643,-1
27644,-1
27645,-1
27646,-1
27647,-1
27648,-1
27649,1
27650,1
27651,1


In [196]:
res.to_csv("./data/pred.csv")

In [106]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=100, 
                                random_state=1,
                                n_jobs=4)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_train)
print('Accuracy (forest): %.2f' % accuracy_score(y_train, y_pred))

y_pred = forest.predict(X_test)
print('Accuracy (forest): %.2f' % accuracy_score(y_test, y_pred))

Accuracy (forest): 1.00
Accuracy (forest): 0.52


In [107]:
importances = forest.feature_importances_
# get sort indices in descending order
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %s %f" % (f + 1, 
                            X.columns.values[indices[f]], 
                            importances[indices[f]]))

# plt.figure()
# plt.title('Feature Importances')
# plt.bar(range(X_train.shape[1]),
#         importances[indices],
#         align='center',
#         alpha=0.5)

 1) article_weight 0.108681
 2) cat_weight 0.105707
 3) nonstopword_rate 0.104649
 4) unique_rate 0.103953
 5) title_freq 0.103008
 6) num_char 0.094433
 7) num_words 0.091193
 8) num_sentence 0.080901
 9) # of total 0.070878
10) # of link 0.068470
11) # of img 0.047597
12) is_mon 0.010797
13) is_weekend 0.009732
